## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-01-12-00 +0000,wsj,Brazil’s Supreme Court Orders Ex-President Bol...,https://www.wsj.com/world/americas/brazils-sup...
1,2025-08-05-01-11-10 +0000,bbc,US condemns house arrest of Brazil's ex-presid...,https://www.bbc.com/news/articles/crkzn5l6jxzo...
2,2025-08-05-01-09-51 +0000,nyt,"After Trump’s Firing of BLS Commissioner, Repu...",https://www.nytimes.com/2025/08/04/us/politics...
3,2025-08-05-01-03-45 +0000,nypost,Clip of Sydney Sweeney’s expert shooting range...,https://nypost.com/2025/08/04/us-news/clip-of-...
4,2025-08-05-01-00-00 +0000,wsj,"Wages have risen, but inflation in food and ho...",https://www.wsj.com/economy/earning-more-worse...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,46
63,new,21
304,will,13
7,house,13
118,texas,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
46,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,123
275,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,102
63,2025-08-04-22-27-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...,102
30,2025-08-04-23-23-00 +0000,wsj,President Trump’s decision to fire the head of...,https://www.wsj.com/economy/real-strains-insid...,96
5,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,91


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
46,123,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
76,58,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
63,56,2025-08-04-22-27-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...
257,46,2025-08-04-08-00-20 +0000,cbc,The nuclear arms race is heating up. Will it t...,https://www.cbc.ca/radio/sunday/hiroshima-naga...
191,46,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
83,42,2025-08-04-21-32-27 +0000,nypost,"Girl, 1, found dead in apparent drowning at NY...",https://nypost.com/2025/08/04/us-news/girl-1-f...
10,41,2025-08-05-00-46-41 +0000,wapo,Netanyahu vows to occupy Gaza as U.S. touts ‘a...,https://www.washingtonpost.com/national-securi...
3,39,2025-08-05-01-03-45 +0000,nypost,Clip of Sydney Sweeney’s expert shooting range...,https://nypost.com/2025/08/04/us-news/clip-of-...
123,36,2025-08-04-19-31-32 +0000,nypost,Trump lambasts ‘Russian war machine’ as specia...,https://nypost.com/2025/08/04/world-news/trump...
96,34,2025-08-04-21-04-00 +0000,wsj,The State Department may require travelers ent...,https://www.wsj.com/politics/policy/trump-admi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
